In [1]:
import numpy as np

# =========================
# Dense Layer
# =========================
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons, weights, biases):
        self.weights = np.array(weights, dtype=float)   # shape (n_inputs, n_neurons)
        self.biases = np.array(biases, dtype=float)     # shape (1, n_neurons)

    def forward(self, inputs):
        self.inputs = np.array(inputs, dtype=float)     # shape (1, n_inputs)
        self.output = np.dot(self.inputs, self.weights) + self.biases

    def backward(self, dvalues, learning_rate=0.1):
        self.dweights = np.dot(self.inputs.T, dvalues)  # grads for weights
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)  # grads for biases
        self.dinputs = np.dot(dvalues, self.weights.T)  # pass gradient backward

        # Update
        self.weights -= learning_rate * self.dweights
        self.biases -= learning_rate * self.dbiases


# =========================
# ReLU Activation
# =========================
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0


# =========================
# Mean Squared Error Loss
# =========================
class Loss_MSE:
    def forward(self, y_pred, y_true):
        return np.mean((y_true - y_pred) ** 2)

    def backward(self, y_pred, y_true):
        samples = len(y_pred)
        self.dinputs = -2 * (y_true - y_pred) / samples


# =========================
# MAIN (Training Loop)
# =========================
np.set_printoptions(precision=3, suppress=True)

# Input (1 sample, 3 features)
X = [[1, 2, 3]]   # <--- simple input

# Target output (scalar)
y_true = [[5]]

# Hidden layer (3 inputs → 2 neurons)
layer1 = Layer_Dense(
    n_inputs=3, n_neurons=2,
    weights=[[0.1, 0.2],   # simple small weights
             [0.3, 0.4],
             [0.5, 0.6]],
    biases=[[0.1, 0.1]]
)

# Output layer (2 inputs → 1 output)
layer2 = Layer_Dense(
    n_inputs=2, n_neurons=1,
    weights=[[0.7],
             [0.8]],
    biases=[[0.1]]
)

activation1 = Activation_ReLU()
loss_fn = Loss_MSE()

# Training loop
epochs = 5
for step in range(1, epochs+1):
    # ---- Forward pass ----
    layer1.forward(X)
    activation1.forward(layer1.output)

    layer2.forward(activation1.output)
    y_pred = layer2.output

    # ---- Loss ----
    loss = loss_fn.forward(y_pred, y_true)

    # Print details
    print(f"\n===== Step {step} =====")
    print(f"Prediction: {y_pred}")
    print(f"Loss: {loss:.4f}")
    print(f"Layer1 Weights:\n{layer1.weights}")
    print(f"Layer1 Biases:\n{layer1.biases}")
    print(f"Layer2 Weights:\n{layer2.weights}")
    print(f"Layer2 Biases:\n{layer2.biases}")

    # ---- Backward pass ----
    loss_fn.backward(y_pred, y_true)
    layer2.backward(loss_fn.dinputs, learning_rate=0.01)
    activation1.backward(layer2.dinputs)
    layer1.backward(activation1.dinputs, learning_rate=0.01)



===== Step 1 =====
Prediction: [[4.03]]
Loss: 0.9409
Layer1 Weights:
[[0.1 0.2]
 [0.3 0.4]
 [0.5 0.6]]
Layer1 Biases:
[[0.1 0.1]]
Layer2 Weights:
[[0.7]
 [0.8]]
Layer2 Biases:
[[0.1]]

===== Step 2 =====
Prediction: [[4.666]]
Loss: 0.1114
Layer1 Weights:
[[0.114 0.216]
 [0.327 0.431]
 [0.541 0.647]]
Layer1 Biases:
[[0.114 0.116]]
Layer2 Weights:
[[0.745]
 [0.856]]
Layer2 Biases:
[[0.119]]

===== Step 3 =====
Prediction: [[4.912]]
Loss: 0.0077
Layer1 Weights:
[[0.119 0.221]
 [0.337 0.442]
 [0.556 0.664]]
Layer1 Biases:
[[0.119 0.121]]
Layer2 Weights:
[[0.761]
 [0.877]]
Layer2 Biases:
[[0.126]]

===== Step 4 =====
Prediction: [[4.98]]
Loss: 0.0004
Layer1 Weights:
[[0.12  0.223]
 [0.34  0.446]
 [0.56  0.668]]
Layer1 Biases:
[[0.12  0.123]]
Layer2 Weights:
[[0.766]
 [0.883]]
Layer2 Biases:
[[0.128]]

===== Step 5 =====
Prediction: [[4.995]]
Loss: 0.0000
Layer1 Weights:
[[0.12  0.223]
 [0.34  0.446]
 [0.561 0.669]]
Layer1 Biases:
[[0.12  0.123]]
Layer2 Weights:
[[0.767]
 [0.884]]
Layer2 Bi